# AI Writing Assistant: Playground

This notebook allows you to step through the core logic of your AI Writing Assistant. You can inspect the intermediate outputs at each stage, from document retrieval to the final prompt construction and generation.

**Instructions:**
1. Make sure you have run `pip install -r requirements.txt` in your `story` virtual environment.
2. Ensure your Ollama application is running in the background.
3. Run the cells sequentially to see how the system works.

## 1. Configuration

Here, we define the core components we'll be using. You can change `YOUR_PROMPT` and `LLM_MODEL` to experiment.

In [ ]:
# --- Your Query ---
YOUR_PROMPT = "Write a short tutorial on how to use git rebase."

# --- Core Components ---
DB_PATH = "db"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "qwen2:7b-instruct-q4_K_M" # Make sure this model is pulled in Ollama

# --- Imports ---
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import box
import yaml

## 2. Load the Knowledge Base

First, we load our vector database, which contains all the processed chunks from your blog posts.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)

print("Vector store loaded successfully.")

## 3. The Retriever

Now, we create a `retriever`. Its job is to take your prompt, search the vector store, and find the most relevant chunks of your own writing. These chunks will be used as stylistic examples.

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # We'll retrieve 3 chunks for this example

print("Retriever created. Ready to search for stylistic context.")

### See the Retrieved Context

Let's test the retriever with your prompt and see what it finds. This is the raw context that will be fed to the LLM to guide its writing style.

In [ ]:
retrieved_docs = retriever.invoke(YOUR_PROMPT)

print(f"Retrieved {len(retrieved_docs)} documents for the prompt: '{YOUR_PROMPT}'
")

for i, doc in enumerate(retrieved_docs):
    print(f"--- Retrieved Document {i+1} ---")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(doc.page_content)
    print("---------------------------
")

## 4. Constructing the Final Prompt

This is the most critical step. We will now assemble the full prompt that gets sent to the LLM. It combines:
1.  **Instructions:** Telling the AI its role (technical vs. creative).
2.  **Context:** The stylistic examples we just retrieved.
3.  **Question:** Your original prompt.

In [ ]:
# Using the same Technical Prompt Template from main.py
TECHNICAL_PROMPT_TEMPLATE = """
**Instructions:** You are a technical writer. Your primary goal is to create a clear, informative, and well-structured technical article based on the user's request.
Adopt the author's writing style from the examples below, focusing on how they explain complex topics.
Prioritize clarity, accuracy, and logical structure. Use the provided context for stylistic guidance only.

**Author's Style Examples (Context):**
---
{context}
---

**User's Request (Question):**
{question}

**Formatting Instructions:**
Use Markdown for formatting, including headings, lists, and bold/italic text where appropriate.

**Your Story:**
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=TECHNICAL_PROMPT_TEMPLATE
)

# Manually create the context string
context_string = "

---

".join([doc.page_content for doc in retrieved_docs])

final_prompt = prompt_template.format(
    context=context_string,
    question=YOUR_PROMPT
)

print("--- FINAL PROMPT SENT TO LLM ---")
print(final_prompt)
print("---------------------------------")

## 5. Generate the Story

Finally, we send the fully constructed prompt to the local LLM and stream the response.

In [ ]:
llm = OllamaLLM(model=LLM_MODEL, stop=["<|begin_of_text|>", "<|end_of_text|>"])

print("--- AI-Generated Story ---")

# Using the final_prompt we constructed above
full_response = llm.invoke(final_prompt)
print(full_response)

print("
--- End of Story ---")